# This notebook will pull the latest copy of your resources into a common workspace (annotation updates are pretty common for NDA; possibly for REDCap, too, depending on activity)

In [ ]:
import os
import nda_yaml
from ccf.redcap import RedcapTable
from ccf.easy_yaml import EasyYaml

Y = EasyYaml()
tags = []

# Download NDA structures

In [ ]:
if os.path.exists('nda') and not os.path.exists('.bk.nda'):
    os.rename('nda', '.bk.nda')

structures = 'phenx_sib01 eatdisdemo01 scidv_pscyh01 diagpsx01 asr01 batbil01 bisbas01 bsc01 cbcl01 cbcl1_501 cbq01 deldisk01 drugscr01 eatq01 edinburgh_hand01 er4001 fenvs01 gales01 gbi01 gonogo01 ipaq01 lbadl01 leap01 mab01 mchq01 mctq01 medh01 mendt01 mmse01 moca01 neo_ffi_form_s_adult_200301 nffi01 pds01 phenx_su01 psqi01 ravlt01 saiq01 scan_debrief01 screentime01 sdq01 sleepdis01 socdem01 srs02 upps01 vision_tests01 vitals01 wais_iv_part101 wisc_v01 wppsiiv01 ysr01'.split(' ')
for struct in structures:
    nda_yaml.download(struct)

if os.path.exists('.bk.nda') and os.path.exists('nda'):
    !!meld .bk.nda/ nda/

In [ ]:
%%time
for filename in os.listdir("nda/"):
    path = f"nda/{filename}"
    contents = Y(path)
    tags.extend([f"{x}\t{path}\t/^{x}:$/;\"\tnda" for x in contents.keys()])

Original time: **user** 14.5 s **sys:** 729 ms  **total:** 15.2 s

# Download Redcap definitions

In [ ]:
if os.path.exists('nda') and not os.path.exists('.bk.definitions'):
    os.rename('definitions', '.bk.definitions')

#name of the databases under 'datasources' in the secrets.yml
#error messages probably indicate that you have a token that is no longer active or not specified
structures = ['hcpa']#,'hcpd18','hcpdchild','hcpdparent','qint','ksads','ssaga','teen','child','parent']
for struct in structures:
    print('trying '+ struct + ' database')
    RedcapTable.get_table_by_name(struct).download_datadictionary('definitions')
    try:
        RedcapTable.get_table_by_name(struct).download_datadictionary('definitions')
    except: 
       print('cound not connect')
if os.path.exists('.bk.definitions') and os.path.exists('definitions'):
    !!meld .bk.definitions/ definitions/

In [ ]:
%%time

for filename in os.listdir("definitions/"):
    path = f"definitions/{filename}"
    contents = Y(path)
    tags.extend([f"{x}\t{path}\t/^{x}:$/;\"\tredcap" for x in contents.keys()])
    

Original time: **user** 20.3 s **sys:** 126 ms  **total:** 20.4 s

# Write Tags
(To make it easy to jump to definitions in the data dictionary yaml files in `vim`.  Kind of like having a hyperlink)

In [ ]:
with open('tags', 'w') as f:
    tags.sort()
    f.write('\n'.join(tags))

In [ ]:
#add redcap_event_name to dictionaries

In [ ]:
import yaml

with open('./definitions/hcpa.yaml','r') as file:
    ddict=yaml.safe_load(file)
    
ddict["redcap_event_name"]={'form':'study_schema','type':'text','label':'REDCap Event Name'}
with open(r'./definitions/hcpa.yaml','w') as file:
    documents=yaml.dump(ddict,file)

In [ ]:
with open('./definitions/hcpa.yaml','r') as file:
    ddict=yaml.safe_load(file)
a=dict(list(ddict.items())[-3:])
a
ddict['redcap_event_name']

In [ ]:
for i in ['hcpd18','hcpdchild','hcpdparent','qint','ksads','ssaga','teen','child','parent']:
    with open('./definitions/'+ i +'.yaml','r') as file:
        ddict=yaml.safe_load(file)

    ddict["redcap_event_name"]={'form':'study_schema','type':'text','label':'REDCap Event Name'}
    with open(r'./definitions/'+ i +'.yaml','w') as file:
        documents=yaml.dump(ddict,file)
